In [14]:
import torch
t = torch.randn(2,2).cuda()
print(t.is_cuda)

csv_path = "/home/shine/PoisoningAttack/dataset/"
csv_name = "from_adgraph/features_rf.csv"
# csv_name = "adgraph_rf_data_encoded.csv"
# save_path = "/home/shine/PoisoningAttack/saved_model/surrogate.pt"


True


In [15]:
import os 
import pandas as pd 
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from sklearn.utils import resample
from joblib import dump, load
import pickle

to_exclude = ["DOMAIN_NAME", "NODE_ID", "FEATURE_KATZ_CENTRALITY", "FEATURE_FIRST_PARENT_KATZ_CENTRALITY", "FEATURE_SECOND_PARENT_KATZ_CENTRALITY"]

os.getcwd()
data = pd.read_csv(csv_path + csv_name)
#data = data.sample(frac=1) #row shuffle
# print(type(data))
# print(data.head())

nCar = data.shape[0]
nVar = data.shape[1]
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

X = data.drop("CLASS", axis=1)
# X = data.drop(to_exclude, axis=1)
y = data["CLASS"]

count_class_0, count_class_1 = data['CLASS'].value_counts()
print("-----")
print("NonAD (before dropping) : {}".format(count_class_0))
print("AD (before dropping) : {}".format(count_class_1))

count_class_0_to_keep = int(count_class_1 * 2.3)

class_0_indices = data[data['CLASS'] == 0].index
random_indices = np.random.choice(class_0_indices, size=count_class_0_to_keep, replace=False)
class_0_sample = data.loc[random_indices]
data_sampled = pd.concat([class_0_sample, data[data['CLASS'] == 1]])

data_sampled = data_sampled.reset_index(drop=True)

X = data_sampled.drop("CLASS", axis=1)
y = data_sampled["CLASS"]
# X = data_sampled.drop(to_exclude, axis=1)

count_class_0, count_class_1 = data_sampled['CLASS'].value_counts()
print("-----")
print("NonAD (after dropping) : {}".format(count_class_0))
print("AD (after dropping) : {}".format(count_class_1))
print("Ratio : {:.0f} : {:.0f}".format(count_class_0 / (count_class_0 + count_class_1) * 100, count_class_1 / (count_class_0 + count_class_1) * 100))


clf = RandomForestClassifier(n_estimators=100, random_state=41)


nCar: 276809 nVar: 71
-----
NonAD (before dropping) : 240921
AD (before dropping) : 35887
-----
NonAD (after dropping) : 82540
AD (after dropping) : 35887
Ratio : 70 : 30


In [16]:
# Define the stratified cross-validation object
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=12)

best_accuracy = 0.0
best_clf = None

accuracy_history = []
precision_history = []
recall_history = []

# For each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print("Fold ", fold)
    
    # if fold != 1:
    #     continue
    
    fold_train_y = y[train_index]
    fold_test_y = y[test_index]
    for class_label in np.unique(y):
        train_count = np.count_nonzero(fold_train_y == class_label)
        test_count = np.count_nonzero(fold_test_y == class_label)
        if fold == 0 or fold == 5 or fold == 9:
            print("Class label:", class_label, "train count:", train_count, "test count:", test_count)

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train = X_train.drop(to_exclude, axis=1)
    X_test = X_test.drop(to_exclude, axis=1)

    # Train
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    accuracy_history.append(accuracy_score(y_test, y_pred))
    precision_history.append(precision_score(y_test, y_pred))
    recall_history.append(recall_score(y_test, y_pred))
    
    if accuracy_score(y_test, y_pred) > best_accuracy:
        best_accuracy = accuracy_score(y_test, y_pred)
        best_clf = clf

print(recall_history)
# Print
print("Average accuracy:", np.mean(accuracy_history))
print("Average precision:", np.mean(precision_history))
print("Average recall:", np.mean(recall_history))

# Save
if best_clf is not None:
    with open('../../saved_model/rf_model_30perc.pt', 'wb') as f:
        pickle.dump(best_clf, f)

Fold  0
Class label: 0.0 train count: 74286 test count: 8254
Class label: 1.0 train count: 32298 test count: 3589
Fold  1
Fold  2
Fold  3
Fold  4
Fold  5
Class label: 0.0 train count: 74286 test count: 8254
Class label: 1.0 train count: 32298 test count: 3589
Fold  6
Fold  7
Fold  8
Fold  9
Class label: 0.0 train count: 74286 test count: 8254
Class label: 1.0 train count: 32299 test count: 3588
[0.8548342156589579, 0.8478684870437447, 0.8581777653942603, 0.8439676790192254, 0.8573418779604347, 0.8528838116466982, 0.8503761493452215, 0.8609253065774805, 0.8595317725752508, 0.8564659977703456]
Average accuracy: 0.9272801382747822
Average precision: 0.900681334074785
Average recall: 0.8542373062991621


In [18]:
# SELECT QUERIES
# variables
query_size = 200000

data = pd.read_csv(csv_path + csv_name)
data = data.sample(frac=1) #row shuffle
X = data.drop("CLASS", axis=1)
y = data["CLASS"]

X = X.drop(to_exclude, axis=1)
# concate
y_result = clf.predict(X)
y_result = pd.DataFrame(y_result, columns=["CLASS"])
y_label = pd.DataFrame(y, columns=["CLASS"])
instances_concat = pd.concat([X, y_result], axis=1)
remaining_concat = pd.concat([X, y_label], axis=1)

# count "all" instances
if query_size == "all":
    query_size = len(instances_concat)
    print("all query size : {}".format(query_size))

# Select random N queries
selcted_query = instances_concat.sample(n=query_size)

# save
if query_size == len(instances_concat):
    query_size = "all"
selcted_query.to_csv("/home/shine/PoisoningAttack/dataset/for_surrogate/query_30perc_{}.csv".format(query_size), index=False)

# save remaining query
selected_indices = selcted_query.index
remaining_queries = remaining_concat.drop(selected_indices)
remaining_queries = remaining_queries[remaining_queries["CLASS"] == 1]
remaining_queries.to_csv("/home/shine/PoisoningAttack/dataset/for_surrogate/query_30perc_{}_for_inversion.csv".format(query_size), index=False)



In [31]:
csv_name_ad = "target_features_rf.csv"
# csv_name_pgd = "perturbed_features_pgd.csv"
# csv_name_nbackdoor = "perturbed_features_nbackdoor.csv"

data_ad = pd.read_csv("/home/shine/PoisoningAttack/dataset/from_adgraph/target_features_rf.csv")
# data_pgd = pd.read_csv(csv_path + csv_name_pgd)
# data_nbackdoor = pd.read_csv(csv_path + csv_name_nbackdoor)

# feature_columns_ad = list(data_ad.columns.difference(['CLASS']))
# feature_columns_pgd = list(data_pgd.columns.difference(['CLASS']))
# feature_columns_nbackdoor = list(data_nbackdoor.columns.difference(['CLASS']))

X_ad = data_ad[feature_columns_ad]
# X_pgd = data_pgd[feature_columns_pgd]
# X_nbackdoor = data_nbackdoor[feature_columns_nbackdoor]

X_ad = data_ad.drop('CLASS', axis=1)
# after_mapping_target_pgd = data_pgd['CLASS']
# after_mapping_target_nbackdoor = data_nbackdoor['CLASS']

y_ad = data_ad['CLASS']

predict_ad = clf.predict(X_ad)
# predict_pgd = clf.predict(X_pgd)
# predict_nbackdoor = clf.predict(X_nbackdoor)

print(accuracy_score(y_ad, predict_ad))
# print(accuracy_score(y_pgd, predict_pgd))
# print(accuracy_score(y_nbackdoor, predict_nbackdoor))



0.619
